<a href="https://colab.research.google.com/github/DatumLearning/Attention-Mechanism/blob/main/Cross_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
num_heads = 8
embedDim = 512
seq_len_q = 10
seq_len_kv = 15
batch_size = 16

In [ ]:
head_dim = embedDim // num_heads
head_dim

64

In [ ]:
query = torch.randn(batch_size , seq_len_q , embedDim)
key = torch.randn(batch_size , seq_len_kv , embedDim)
value = torch.randn(batch_size , seq_len_kv , embedDim)

In [ ]:
query.shape , key.shape , value.shape

(torch.Size([16, 10, 512]),
 torch.Size([16, 15, 512]),
 torch.Size([16, 15, 512]))

In [ ]:
q_linear = nn.Linear(embedDim , embedDim)
k_linear = nn.Linear(embedDim , embedDim)
v_linear = nn.Linear(embedDim , embedDim)

In [ ]:
Q = q_linear(query)
K = k_linear(key)
V = v_linear(value)

In [ ]:
Q.shape , K.shape , V.shape

(torch.Size([16, 10, 512]),
 torch.Size([16, 15, 512]),
 torch.Size([16, 15, 512]))

In [ ]:
Qv = Q.view(batch_size , -1 , num_heads , head_dim)
Kv = K.view(batch_size , -1 , num_heads , head_dim)
Vv = V.view(batch_size , -1 , num_heads , head_dim)

In [ ]:
Qv.shape , Kv.shape , Vv.shape

(torch.Size([16, 10, 8, 64]),
 torch.Size([16, 15, 8, 64]),
 torch.Size([16, 15, 8, 64]))

In [ ]:
Qt = Qv.transpose(1 , 2)
Kt = Kv.transpose(1 , 2)
Vt = Vv.transpose(1 , 2)

In [ ]:
Qt.shape , Kt.shape , Vt.shape

(torch.Size([16, 8, 10, 64]),
 torch.Size([16, 8, 15, 64]),
 torch.Size([16, 8, 15, 64]))

In [ ]:
scores = torch.matmul(Qt , Kt.transpose(-2 , -1)) / (head_dim ** 0.5)
scores.shape

torch.Size([16, 8, 10, 15])

In [ ]:
attention_weights = F.softmax(scores , dim = -1)
attention_weights.shape

torch.Size([16, 8, 10, 15])

In [ ]:
output = torch.matmul(attention_weights , Vt)
output.shape

torch.Size([16, 8, 10, 64])

In [ ]:
output = output.transpose(1 , 2).contiguous().view(batch_size , -1 , embedDim)
output.shape

torch.Size([16, 10, 512])